In [1]:
#!pip install -r requirements.txt

from IPython import display
display.clear_output()

In [2]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
from collections import Counter
import logging as log
import json

log.basicConfig(level=log.INFO)

WARNING ⚠️ Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at '/Users/huraira/Library/Application Support/Ultralytics/settings.yaml'


In [3]:

model = YOLO("yolov8m.pt")
label_map = model.names


In [4]:
import supervision as sv

# Load in our sample video
VID_PATH = "/Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4"

# Show the dimensions and additional information from the video
video_info = sv.VideoInfo.from_video_path(VID_PATH)
print(video_info)


VideoInfo(width=3840, height=2160, fps=29, total_frames=640)


In [5]:
#Extract a single frame from the video
generator = sv.get_video_frames_generator(VID_PATH)
iterator = iter(generator)
frame = next(iterator)
#Save the frame
cv2.imwrite("frame.jpg", frame)

True

In [6]:
def load_zones(json_path, zone_str):
    """
        Load zones specified in an external json file
        Parameters:
            json_path: path to the json file with defined zones
            zone_str:  name of the zone in the json file
        Returns:
           zones: a list of arrays with zone points
    """
    # load json file
    with open(json_path) as f:
        zones_dict = json.load(f)
    # return a list of zones defined by points
    return np.array(zones_dict[zone_str]["points"], np.int32)

In [7]:
polygon = load_zones("zones.json", "test-example-1")
polygon

array([[ 776,  321],
       [3092,  305],
       [3112, 1965],
       [ 596, 2005],
       [ 768,  321]], dtype=int32)

In [8]:
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

In [9]:
def draw_text(image, text, point, color=(255, 255, 255)) -> None:
    """
    Draws text

    Parameters:
        image: image to draw on
        text: text to draw
        point:
        color: text color
    """
    _, f_width = image.shape[:2]
    
    text_size, _ = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, thickness=2)

    rect_width = text_size[0] + 20
    rect_height = text_size[1] + 20
    rect_x, rect_y = point

    cv2.rectangle(image, pt1=(rect_x, rect_y), pt2=(rect_x + rect_width, rect_y + rect_height), color=(255, 255, 255), thickness=cv2.FILLED)

    text_x = (rect_x + (rect_width - text_size[0]) // 2) - 10
    text_y = (rect_y + (rect_height + text_size[1]) // 2) - 10
    
    cv2.putText(image, text=text, org=(text_x, text_y), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=color, thickness=2, lineType=cv2.LINE_AA)

In [10]:
def get_iou(person_det, object_det):
    #Obtain the Intersection 
    x_left = max(person_det[0], object_det[0])
    y_top = max(person_det[1], object_det[1])
    x_right = min(person_det[2], object_det[2])
    y_bottom = min(person_det[3], object_det[3])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    person_area = (person_det[2] - person_det[0]) * (person_det[3] - person_det[1])
    obj_area = (object_det[2] - object_det[0]) * (object_det[3] - object_det[1])
    
    return intersection_area / float(person_area + obj_area - intersection_area)

In [11]:
def intersecting_bboxes(bboxes, person_bbox, action_str):
    #Identify if person and object bounding boxes are intersecting using IOU
    for box in bboxes:
      if box.cls == 0:
          #If it is a person
          try:
              person_bbox.append([box.xyxy[0], box.id.numpy().astype(int)])
          except:
              pass
      elif box.cls != 0 and len(person_bbox) >= 1:
          #If it is not a person and an interaction took place with a person
          for p_bbox in person_bbox:
              if box.cls != 0:
                  result_iou = get_iou(p_bbox[0], box.xyxy[0])
                  if result_iou > 0:
                     try:
                        person_intersection_str = f"Person #{p_bbox[1][0]} interacted with object #{int(box.id[0])} {label_map[int(box.cls[0])]}"
                     except:
                         person_intersection_str = f"Person {p_bbox[1][0]} interacted with object (ID unable to be assigned) {label_map[int(box.cls[0])]}"
                     #log.info(person_intersection_str)
                     person_action_str = action_str + f" by person {p_bbox[1][0]}"
                     return person_action_str

In [12]:
pip install firebase-admin



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import firebase_admin
from firebase_admin import credentials, db
# Reinitialize a new Firebase app with a unique name
cred = credentials.Certificate('/Users/huraira/Desktop/Module3/openvino_notebooks/fbis-19de1-firebase-adminsdk-idgon-e51fea33fe.json')
firebase_admin.initialize_app(cred, {
    "databaseURL": "https://fbis-19de1-default-rtdb.firebaseio.com/"
}, name="new_app")
# Use the new app explicitly when creating a database reference
new_app = firebase_admin.get_app("new_app")
receipts_ref = db.reference("receipts", app=new_app)
# Fetch data
print(receipts_ref.get())  # Example: Retrieve data at "receipts"

ValueError: Firebase app named "new_app" already exists. This means you called initialize_app() more than once with the same app name as the second argument. Make sure you provide a unique name every time you call initialize_app().

In [18]:
#Define empty lists to keep track of labels
original_labels = []
final_labels = []
person_bbox = []
p_items = []
purchased_items = set(p_items)
a_items = []
added_items = set(a_items)

# Dictionary to store persistent tracking state of objects
object_tracking_state = {}
#Save result as det_tracking_result
with sv.VideoSink("new_det_tracking_result2.mp4", video_info) as sink:
    #Iterate through model predictions and tracking results
    for index, result in enumerate(model.track(source=VID_PATH, show=False, classes =[0,39,46,47], tracker = "bytetrack.yaml", stream=True, verbose=True, persist=True)):
      #Define variables to store interactions that are refreshed per frame
      interactions = []
      person_intersection_str = ""

      #Obtain predictions from yolov8 model
      frame = result.orig_img
      detections = sv.Detections.from_ultralytics(result)
      detections = detections[detections.class_id < 55]
      mask = zone.trigger(detections=detections)
      detections_filtered = detections[mask]
      bboxes = result.boxes
      
      # Ensure tracking IDs are assigned and filter for recognized IDs if any are specified
      if bboxes.id is not None:
          detections.tracker_id = bboxes.id.cpu().numpy().astype(int)
          
          # If recognized_ids is not empty, filter detections
    
      labels = [
          f'#{tracker_id} {label_map[class_id]} {confidence:0.2f}'
          for _, _, confidence, class_id, tracker_id
          in detections
      ]

      #Annotate the frame with the zone and bounding boxes.
      frame = box_annotator.annotate(scene=frame, detections=detections_filtered, labels=labels)
      frame = zone_annotator.annotate(scene=frame)

      # Create objects list with tracking IDs and class names
      objects = [f'#{tracker_id} {label_map[class_id]}' for _, _, _, class_id, tracker_id in detections]
      
      # Track current frame's objects
      current_frame_objects = {
          f'#{tracker_id} {label_map[class_id]}': label_map[class_id] 
          for _, _, _, class_id, tracker_id in detections
      }

      #If this is the first time we run the application,
      #store the objects' labels as they are at the beginning
      if index == 0:
          original_labels = objects
          original_dets = len(detections_filtered)
          # Initialize tracking state for first frame
          for obj in objects:
              object_tracking_state[obj] = True
          continue

      #To identify if an object has been added or removed
      #we'll use the original labels and identify any changes
      final_labels = objects
      new_dets = len(detections_filtered)
      
      #Create two variables we can increment for drawing text
      draw_txt_ir = 1
      draw_txt_ia = 1
      
      # Update tracking state for existing objects
      for obj in original_labels:
          # Mark objects as not in current frame if they're no longer detected
          if obj not in final_labels:
              object_tracking_state[obj] = False

      # Check for objects being added or removed
      for obj in original_labels:
          # Check if object has been removed from the frame
          if not object_tracking_state.get(obj, False):
              # Ignore person objects
              if 'person' not in obj:
                  # Extract class name (without tracking ID)
                  class_name = obj.split(' ', 2)[-1]
                  
                  removed_object_str = f"1 {class_name} removed from zone"
                  removed_action_str = intersecting_bboxes(bboxes, person_bbox, removed_object_str)
                  
                  if removed_action_str is not None:
                      log.info(removed_action_str)
                      # Add to purchased items using class name
                      if class_name not in purchased_items:
                          purchased_items.add(class_name)
                          p_items.append(f" - {class_name}") 
                      # Draw the result on the screen        
                      draw_text(frame, text=removed_action_str, point=(50, 50 + draw_txt_ir), color=(0, 0, 255))
                      draw_txt_ir += 80

      # Check for newly added objects
      for obj in final_labels:
          # Check if this is a new object or newly visible object
          if obj not in original_labels:
              # Ignore person objects
              class_name = obj.split(' ', 2)[-1]
              
              if 'person' not in class_name:
                  added_object_str = f"1 {class_name} added to zone"
                  added_action_str = intersecting_bboxes(bboxes, person_bbox, added_object_str)
                  
                  if added_action_str is not None:
                      log.info(added_action_str)
                      # Remove from purchased items if it was previously purchased
                      if class_name in purchased_items:
                          purchased_items.remove(class_name)
                          
                            
                      # Draw the result on the screen  
                      draw_text(frame, text=added_action_str, point=(50, 300 + draw_txt_ia), color=(0, 128, 0))
                      draw_txt_ia += 80

      # Update tracking state for current frame objects
      for obj in final_labels:
          object_tracking_state[obj] = True
      
      # Update original labels for next iteration
      original_labels = final_labels

      original_dets = new_dets
      receipt_data = {
        "yZkhYZJTlaXB03jrBXNHUthsvoN2": {  # You can dynamically change the person ID as needed
           "items": list(purchased_items),
           "timestamp": index  # Use frame index or you could use a proper timestamp
    }
}

      
      receipts_ref.set(receipt_data)
      
      draw_text(frame, "Receipt: " + str(purchased_items), point=(50, 800), color=(30, 144, 255))
      sink.write_frame(frame)


video 1/1 (1/640) /Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4: 384x640 1 bottle, 1 apple, 1164.7ms
video 1/1 (2/640) /Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4: 384x640 1 bottle, 1 banana, 1 apple, 1151.8ms
video 1/1 (3/640) /Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4: 384x640 1 bottle, 1 banana, 1 apple, 1137.6ms
video 1/1 (4/640) /Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4: 384x640 1 bottle, 1 banana, 1 apple, 1149.7ms
video 1/1 (5/640) /Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4: 384x640 1 bottle, 1 banana, 1 apple, 1168.0ms
video 1/1 (6/640) /Users/huraira/Desktop/Module3/openvino_notebooks/recipes/automated_self_checkout/data/example.mp4: 384x640 1 bottle, 1 banana, 1 apple, 1138.1ms
video 1/1 (7/640) /Users/

In [30]:
"Receipt: " + str(purchased_items)

"Receipt: {'apple'}"

In [32]:
added_objects

Counter({'#145 bottle': 1, '#154 apple': 1, '#155 banana': 1})